In [11]:
#!pip install matplotlib
#!pip install tensorflow


# Constucturing The Model

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
import pandas as pd


# Constucturing The Model

In [4]:
from tensorflow.python.keras.layers import Dense, Flatten


In [5]:
dnn_model = Sequential()

imported_model = tf.keras.applications.ResNet50(
    include_top=False,
    input_shape=(180, 180, 3), # 180,180 remeber this number
    pooling='avg',
    classes=10,  # 10 represent num of labels 
    weights='imagenet')

# Freeze all layers in the imported model
for layer in imported_model.layers:
    layer.trainable = False

# Check if the layers are trainable
for layer in imported_model.layers:
    print(f"Layer {layer.name}: Trainable = {layer.trainable}")


dnn_model.add(imported_model)
dnn_model.add(Flatten())
dnn_model.add(Dense(600, activation='relu'))
dnn_model.add(Dense(10, activation='softmax')) # 10 represent num of labels 



#from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.legacy import Adam
dnn_model.compile(optimizer=Adam(lr=0.001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

Layer input_1: Trainable = False
Layer conv1_pad: Trainable = False
Layer conv1_conv: Trainable = False
Layer conv1_bn: Trainable = False
Layer conv1_relu: Trainable = False
Layer pool1_pad: Trainable = False
Layer pool1_pool: Trainable = False
Layer conv2_block1_1_conv: Trainable = False
Layer conv2_block1_1_bn: Trainable = False
Layer conv2_block1_1_relu: Trainable = False
Layer conv2_block1_2_conv: Trainable = False
Layer conv2_block1_2_bn: Trainable = False
Layer conv2_block1_2_relu: Trainable = False
Layer conv2_block1_0_conv: Trainable = False
Layer conv2_block1_3_conv: Trainable = False
Layer conv2_block1_0_bn: Trainable = False
Layer conv2_block1_3_bn: Trainable = False
Layer conv2_block1_add: Trainable = False
Layer conv2_block1_out: Trainable = False
Layer conv2_block2_1_conv: Trainable = False
Layer conv2_block2_1_bn: Trainable = False
Layer conv2_block2_1_relu: Trainable = False
Layer conv2_block2_2_conv: Trainable = False
Layer conv2_block2_2_bn: Trainable = False
Layer co

/Users/EdenChung/anaconda3/envs/ml/lib/python3.9/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [6]:
#dnn_model.load_weights('eden_model_weights.h5')
dnn_model.load_weights('resnet_no_augmentation_batch20.h5')


In [15]:
valid_path = 'valid'

In [9]:
height,width=180,180



In [16]:
batches=10
valid_data = tf.keras.preprocessing.image_dataset_from_directory(valid_path,image_size=(height,width))
valid_data = valid_data.take(batches)

Found 10244 files belonging to 10 classes.


In [9]:
import tensorflow as tf


for images, labels in test_data.take(1):  
    
    images_resized = tf.image.resize(images, (height, width))
    predictions = dnn_model.predict(images_resized)


    predicted_labels = tf.argmax(predictions, axis=1) # Logit 

    predicted_labels_numpy = predicted_labels.numpy()

    for i in range(14):
        print(f"Predicted label for image {i + 1}: {predicted_labels_numpy[i]}",f" actual label for image {i + 1}: {labels[i]}")
   



1/1 [==============================] - 1s 945ms/step
Predicted label for image 1: 9  actual label for image 1: 2
Predicted label for image 2: 2  actual label for image 2: 0
Predicted label for image 3: 2  actual label for image 3: 0
Predicted label for image 4: 2  actual label for image 4: 2
Predicted label for image 5: 9  actual label for image 5: 5
Predicted label for image 6: 2  actual label for image 6: 4
Predicted label for image 7: 1  actual label for image 7: 1
Predicted label for image 8: 2  actual label for image 8: 0
Predicted label for image 9: 5  actual label for image 9: 4
Predicted label for image 10: 2  actual label for image 10: 3
Predicted label for image 11: 1  actual label for image 11: 0
Predicted label for image 12: 0  actual label for image 12: 0
Predicted label for image 13: 2  actual label for image 13: 0
Predicted label for image 14: 5  actual label for image 14: 4


In [22]:
#full evaluation metrics
all_predictions = []
all_labels = []

for images, labels in valid_data:
    images_resized = tf.image.resize(images, (height, width))
    predictions = dnn_model.predict(images_resized)
    predicted_labels = tf.argmax(predictions, axis=1)

    all_predictions.extend(predicted_labels.numpy())
    all_labels.extend(labels.numpy())

all_predictions = np.array(all_predictions)
all_labels = np.array(all_labels)

accuracy = np.mean(all_predictions == all_labels)
print(f"Accuracy: {accuracy * 100:.2f}%")



1/1 [==============================] - 1s 638ms/step
Accuracy: 88.75%


## Create the submission

In [11]:
##Creating the submission

#test_df = pd.read_csv('test.csv')
test_df = pd.read_csv('BTTAIxNYBG-test.csv').head(5)

# Initialize a list to store predictions
predictions_list = []

for _, row in test_df.iterrows():
    unique_id = row['uniqueID']
    file_name = row['imageFile']
    
    # Load and preprocess the image
    img_path = os.path.join('Test_before', file_name)  # Update with your test images path
    img = image.load_img(img_path, target_size=(height, width))
    img_array = image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create a batch dimension
    
    # Predict
    predictions = dnn_model.predict(img_array)
    predicted_class = tf.argmax(predictions, axis=1).numpy()[0]  # Take the argmax to get the predicted class ID
    
    # Store the uniqueID and predicted classID
    predictions_list.append({'uniqueID': unique_id, 'classID': predicted_class})

# Convert the predictions list to a DataFrame
predictions_df = pd.DataFrame(predictions_list)

# Save the predictions to a new CSV file
predictions_df.to_csv('predictions.csv', index=False)


1/1 [==============================] - 0s 193ms/step


In [12]:
dnn_model = Sequential()

imported_model = tf.keras.applications.ResNet50(
    include_top=False,
    input_shape=(180, 180, 3),
    pooling='avg',
    classes=10,
    weights='imagenet')

# Freeze all layers in the imported model
for layer in imported_model.layers:
    layer.trainable = False

# Check if the layers are trainable
for layer in imported_model.layers:
    print(f"Layer {layer.name}: Trainable = {layer.trainable}")


dnn_model.add(imported_model)
dnn_model.add(Flatten())
#dnn_model.add(Dense(600, activation='relu'))
#could go back to 600 later
dnn_model.add(Dense(256, activation='relu'))
dnn_model.add(Dense(10, activation='softmax'))



#from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.legacy import Adam
dnn_model.compile(optimizer=Adam(lr=0.001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
dnn_model.load_weights('resnet_no_augmentation_full_dataset_epochs10.h5')



Layer input_2: Trainable = False
Layer conv1_pad: Trainable = False
Layer conv1_conv: Trainable = False
Layer conv1_bn: Trainable = False
Layer conv1_relu: Trainable = False
Layer pool1_pad: Trainable = False
Layer pool1_pool: Trainable = False
Layer conv2_block1_1_conv: Trainable = False
Layer conv2_block1_1_bn: Trainable = False
Layer conv2_block1_1_relu: Trainable = False
Layer conv2_block1_2_conv: Trainable = False
Layer conv2_block1_2_bn: Trainable = False
Layer conv2_block1_2_relu: Trainable = False
Layer conv2_block1_0_conv: Trainable = False
Layer conv2_block1_3_conv: Trainable = False
Layer conv2_block1_0_bn: Trainable = False
Layer conv2_block1_3_bn: Trainable = False
Layer conv2_block1_add: Trainable = False
Layer conv2_block1_out: Trainable = False
Layer conv2_block2_1_conv: Trainable = False
Layer conv2_block2_1_bn: Trainable = False
Layer conv2_block2_1_relu: Trainable = False
Layer conv2_block2_2_conv: Trainable = False
Layer conv2_block2_2_bn: Trainable = False
Layer co

/Users/EdenChung/anaconda3/envs/ml/lib/python3.9/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [13]:
##Creating the submission

#test_df = pd.read_csv('test.csv')
test_df = pd.read_csv('BTTAIxNYBG-test.csv')

# Initialize a list to store predictions
predictions_list = []

for _, row in test_df.iterrows():
    unique_id = row['uniqueID']
    file_name = row['imageFile']
    
    # Load and preprocess the image
    img_path = os.path.join('Test_before', file_name)  # Update with your test images path
    img = image.load_img(img_path, target_size=(height, width))
    img_array = image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create a batch dimension
    
    # Predict
    predictions = dnn_model.predict(img_array)
    predicted_class = tf.argmax(predictions, axis=1).numpy()[0]  # Take the argmax to get the predicted class ID
    
    # Store the uniqueID and predicted classID
    predictions_list.append({'uniqueID': unique_id, 'classID': predicted_class})

# Convert the predictions list to a DataFrame
predictions_df = pd.DataFrame(predictions_list)

# Save the predictions to a new CSV file
predictions_df.to_csv('predictions.csv', index=False)



1/1 [==============================] - 0s 90ms/step
